Brian Romero

Diego Hidalgo

In [ ]:
!pip install contractions

In [ ]:
!pip install clean-text


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from wordcloud import WordCloud
import string
from sklearn.model_selection import train_test_split
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.pipeline import Pipeline
import contractions
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from cleantext import clean

#métricas
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
import time

#Gráficos
import seaborn as sns
import matplotlib.pyplot as plt

#Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA


sns.set_theme(style="whitegrid")

<ipython-input-4-a2adc8f99985>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [ ]:
import warnings
warnings.filterwarnings('ignore')
tqdm_notebook().pandas()

0it [00:00, ?it/s]

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
df = pd.read_csv("/gdrive/MyDrive/IA-PROYECTO-2023-1/cyberbullying_tweets.csv")

In [ ]:
df.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [ ]:
df[df.columns] = df[df.columns].astype('string')

In [ ]:
def tweets_cleaner(tweet):
    # remover las URLs
    tweet = re.sub(r'((www.\S+)|(https?://\S+))', r'', tweet)
    # remover números
    tweet = re.sub(r'[0-9]\S+', r'', tweet)
    # remover etiquetas
    tweet = re.sub(r'(@\S+) | (#\S+)', r'', tweet)
    # remover puntuación
    tweet = [char for char in tweet if char not
                                in string.punctuation]
    # convertir a string
    tweet = "".join(tweet)
    # quitar contracciones
    tweet = contractions.fix(tweet)
    # quitar dobles espacios
    tweet = re.sub("\s\s+" , " ", tweet)
    # conjunto de stopwords
    stop_words = set(stopwords.words("english"))
    # remover las stopwords
    tweet = [w for w in tweet.split()
                              if w.lower() not in stop_words]
    tweet = clean(tweet, no_emoji=True)
    return tweet

In [ ]:
df['type_coded'] = df['cyberbullying_type'].astype('category')
df['type_coded'] = df['type_coded'].cat.codes
df.head()

,tweet_text,cyberbullying_type,type_coded
0,"In other words #katandandre, your food was cra...",not_cyberbullying,3
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,3
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,3
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,3
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,3


In [ ]:
df.groupby(by=['type_coded'])['cyberbullying_type'].first()

type_coded
0                    age
1              ethnicity
2                 gender
3      not_cyberbullying
4    other_cyberbullying
5               religion
Name: cyberbullying_type, dtype: string

Por hacer:
- Hacer que todas las categorias tengan la misma cantidad de tweets: reduciendo a la cantidad del menor o aumentar las demás por sampling
- Luego, extraer componentes por PCA

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
                                                df['tweet_text'],
                                                df["type_coded"],
                                                test_size=0.2,
                                                random_state=42)

#x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

In [ ]:
t_algo = TfidfVectorizer(max_features=15000, dtype='uint8')
x_train_tfidf = t_algo.fit_transform(x_train)
x_test_tfidf = t_algo.transform(x_test)
chi2score = chi2(x_train_tfidf, y_train)[0]

In [ ]:
chi2score = chi2(x_train_tfidf, y_train)[0]
chi2score

array([2.22044778, 8.31849852, 4.74718976, ..., 3.40290057, 6.66650375,
       7.85073115])

In [ ]:
x_train_tfidf

<38153x15000 sparse matrix of type '<class 'numpy.float64'>'
	with 739348 stored elements in Compressed Sparse Row format>

In [ ]:
y_train

27942    4
13988    2
30197    4
47342    1
43358    1
        ..
11284    2
44732    1
38158    0
860      3
15795    2
Name: type_coded, Length: 38153, dtype: int8

# **One Hot Encoding**

In [ ]:
df_en = pd.concat([df.drop('cyberbullying_type', axis = 1), pd.get_dummies(df['cyberbullying_type'])], axis=1)

In [ ]:
df_en

,tweet_text,type_coded,age,ethnicity,gender,not_cyberbullying,other_cyberbullying,religion
0,"In other words #katandandre, your food was cra...",3,0,0,0,1,0,0
1,Why is #aussietv so white? #MKR #theblock #ImA...,3,0,0,0,1,0,0
2,@XochitlSuckkks a classy whore? Or more red ve...,3,0,0,0,1,0,0
3,"@Jason_Gio meh. :P thanks for the heads up, b...",3,0,0,0,1,0,0
4,@RudhoeEnglish This is an ISIS account pretend...,3,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...
47687,"Black ppl aren't expected to do anything, depe...",1,0,1,0,0,0,0
47688,Turner did not withhold his disappointment. Tu...,1,0,1,0,0,0,0
47689,I swear to God. This dumb nigger bitch. I have...,1,0,1,0,0,0,0
47690,Yea fuck you RT @therealexel: IF YOURE A NIGGE...,1,0,1,0,0,0,0


In [ ]:
x_train_en, x_test_en, y_train_en, y_test_en = train_test_split(
                                                df['tweet_text'],
                                                df_en.drop(["tweet_text", "type_coded"], axis=1),
                                                test_size=0.2,
                                                random_state=42)

In [ ]:
x_train_en

27942    RT @avatastic: @freebsdgirl Also, anyone who h...
13988    @Questionnier is dis a joke gay rape or babe rape
30197    RT @DavidCleevely: Raspberry Pi Foundation loo...
47342    STOLEN // reply and ill give you one = luv ur ...
43358    People in Navarre are so fucking dumb there wa...
                               ...                        
11284    females like roseanne barr and those 2 hideous...
44732    Itsssss a question dumb telly tubby IS YA CLIT...
38158    One kid turned into a leftist JUST LIKE my bul...
860      RT @RavenHUWolf: University of #Texas-#Arlingt...
15795    @DoctrineDark798 You should be able to save fr...
Name: tweet_text, Length: 38153, dtype: string

In [ ]:
x_train_en.shape

(38153,)

In [ ]:
y_train_en

,age,ethnicity,gender,not_cyberbullying,other_cyberbullying,religion
27942,0,0,0,0,1,0
13988,0,0,1,0,0,0
30197,0,0,0,0,1,0
47342,0,1,0,0,0,0
43358,0,1,0,0,0,0
...,...,...,...,...,...,...
11284,0,0,1,0,0,0
44732,0,1,0,0,0,0
38158,1,0,0,0,0,0
860,0,0,0,1,0,0


In [ ]:
y_train_en.shape

(38153, 6)

# **Keras**

In [ ]:
seed = 7
np.random.seed(seed)
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.preprocessing import sequence

In [ ]:
x_train_tfidf.shape

(38153, 15000)

In [ ]:
x_train_tfidf.toarray().shape

(38153, 15000)

# **Testing Model With Keras - With One Hot**

In [ ]:
model_en = Sequential()
model_en.add(Dense(256, activation='relu', input_shape=(15000,)))
model_en.add(Dropout(0.5))
model_en.add(Dense(6, activation='softmax'))
model_en.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_en.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 256)               3840256   
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_3 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 3,841,798
Trainable params: 3,841,798
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model_en.fit(x_train_tfidf.toarray(), y_train_en, epochs=30, batch_size=512)

Epoch 1/30
75/75 [==============================] - 22s 274ms/step - loss: 0.4991 - accuracy: 0.5209
Epoch 2/30
75/75 [==============================] - 19s 250ms/step - loss: 0.2981 - accuracy: 0.7541
Epoch 3/30
75/75 [==============================] - 16s 219ms/step - loss: 0.1929 - accuracy: 0.8293
Epoch 4/30
75/75 [==============================] - 16s 218ms/step - loss: 0.1475 - accuracy: 0.8575
Epoch 5/30
75/75 [==============================] - 16s 218ms/step - loss: 0.1245 - accuracy: 0.8749
Epoch 6/30
75/75 [==============================] - 17s 224ms/step - loss: 0.1088 - accuracy: 0.8891
Epoch 7/30
75/75 [==============================] - 17s 226ms/step - loss: 0.0981 - accuracy: 0.8973
Epoch 8/30
75/75 [==============================] - 17s 226ms/step - loss: 0.0894 - accuracy: 0.9055
Epoch 9/30
75/75 [==============================] - 17s 219ms/step - loss: 0.0827 - accuracy: 0.9119
Epoch 10/30
75/75 [==============================] - 16s 218ms/step - loss: 0.0768 - accura

In [ ]:
results_en = model_en.evaluate(x_test_tfidf.toarray(), y_test_en)

299/299 [==============================] - 3s 10ms/step - loss: 0.1968 - accuracy: 0.8049


In [ ]:
results_en

[0.19680438935756683, 0.8049061894416809]

# **Testing Model With Keras - No One Hot**

In [ ]:
y_train.shape

(38153,)

In [ ]:
y_train.shape

(38153,)

In [ ]:
y_train

27942    4
13988    2
30197    4
47342    1
43358    1
        ..
11284    2
44732    1
38158    0
860      3
15795    2
Name: type_coded, Length: 38153, dtype: int8

In [ ]:
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(15000,)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='relu'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               3840256   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 3,840,513
Trainable params: 3,840,513
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train_tfidf.toarray(), y_train, epochs=30, batch_size=512)

Epoch 1/30
75/75 [==============================] - 28s 339ms/step - loss: -10.6101 - accuracy: 0.1712
Epoch 2/30
75/75 [==============================] - 17s 230ms/step - loss: -22.5355 - accuracy: 0.1740
Epoch 3/30
75/75 [==============================] - 16s 220ms/step - loss: -23.1346 - accuracy: 0.1823
Epoch 4/30
75/75 [==============================] - 21s 279ms/step - loss: -23.6341 - accuracy: 0.2102
Epoch 5/30
75/75 [==============================] - 17s 233ms/step - loss: -24.1875 - accuracy: 0.2628
Epoch 6/30
75/75 [==============================] - 16s 220ms/step - loss: -24.4412 - accuracy: 0.2858
Epoch 7/30
75/75 [==============================] - 16s 219ms/step - loss: -24.7735 - accuracy: 0.2855
Epoch 8/30
75/75 [==============================] - 17s 231ms/step - loss: -24.8753 - accuracy: 0.2977
Epoch 9/30
75/75 [==============================] - 17s 231ms/step - loss: -24.9069 - accuracy: 0.3035
Epoch 10/30
75/75 [==============================] - 17s 229ms/step - los

In [ ]:
results = model.evaluate(x_test_tfidf.toarray(), y_test)

299/299 [==============================] - 6s 18ms/step - loss: -24.6291 - accuracy: 0.2965


In [ ]:
results